In [1]:
from datetime import datetime, timedelta
import MetaTrader5 as mt5
import plotly.express as pt
from atj_trading.backtester import get_ohlc_history, create_price_fig, Backtester, evaluate_backtest

In [21]:
mt5.initialize()

symbol = 'US500m'
start_dt = datetime.now() - timedelta(days=365*1)
end_dt = datetime.now()

#get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_H1, start_dt, end_dt)
display(ohlc)


sr_period = 24*5
atr_period = sr_period

ohlc['gain'] = ohlc['open'].shift(-1) - ohlc['open']
ohlc['perc_gain'] = (ohlc['gain']/ohlc['open'])*100

ohlc['high_rounded'] = ohlc['high'].round(0)
ohlc['low_rounded']  = ohlc['low'].round(0)
ohlc['close_rounded']= ohlc['close'].round(0)

ohlc['prev_high'] = ohlc['high'].shift(1)
ohlc['prev_low']  = ohlc['low'].shift(1)

ohlc['hour'] = ohlc['time'].dt.hour
ohlc['range'] = ohlc['high'] - ohlc['low']

#sma for know about trend
ohlc['sma_10'] = ohlc['open'].rolling(10).mean()
ohlc['sma_200'] = ohlc['open'].rolling(200).mean()

ohlc['trend'] = ohlc.apply(lambda x: 'uptrend' if x['sma_10'] > x['sma_200'] else 'downtrend', axis=1)

ohlc['period_high'] = ohlc['close_rounded'].rolling(sr_period).max().shift(2)
ohlc['period_low']  = ohlc['close_rounded'].rolling(sr_period).min().shift(2)

ohlc['sr_range']    = ohlc['period_high'] - ohlc['period_low']
ohlc['atr']         = (ohlc['high']-ohlc['low']).rolling(atr_period).mean()

ohlc = ohlc.dropna()
ohlc['rel_sr_range']= (ohlc['sr_range']/ohlc['atr']).astype(int)
ohlc['rel_candle_size'] = (ohlc['range']/ohlc['atr']).astype(int).shift(1)

display(ohlc)


fig = create_price_fig(ohlc, indicators=['period_high', 'period_low', 'sma_10', 'sma_200'])
display(fig)

atr_fig = pt.line(ohlc, x='time', y='rel_sr_range')
display(atr_fig)

,time,open,high,low,close
0,2023-10-25 18:00:00,4192.03,4197.92,4186.39,4191.90
1,2023-10-25 19:00:00,4191.89,4199.47,4187.24,4191.52
2,2023-10-25 20:00:00,4192.08,4204.26,4190.83,4192.20
3,2023-10-25 22:00:00,4178.50,4179.34,4172.49,4176.66
4,2023-10-25 23:00:00,4176.65,4177.68,4168.44,4173.46
...,...,...,...,...,...
5845,2024-10-24 13:00:00,5830.15,5834.30,5810.67,5818.39
5846,2024-10-24 14:00:00,5818.25,5824.02,5809.89,5812.82
5847,2024-10-24 15:00:00,5812.93,5818.56,5795.13,5805.15
5848,2024-10-24 16:00:00,5805.31,5814.80,5790.41,5813.65


,time,open,high,low,close,gain,perc_gain,high_rounded,low_rounded,close_rounded,...,range,sma_10,sma_200,trend,period_high,period_low,sr_range,atr,rel_sr_range,rel_candle_size
199,2023-11-07 13:00:00,4364.44,4370.58,4363.12,4370.58,5.99,0.137246,4371.0,4363.0,4371.0,...,7.46,4363.647,4235.95695,uptrend,4377.0,4154.0,223.0,8.590333,25,NaN
200,2023-11-07 14:00:00,4370.43,4374.78,4362.31,4372.71,2.02,0.046220,4375.0,4362.0,4373.0,...,12.47,4364.182,4236.84895,uptrend,4377.0,4160.0,217.0,8.653583,25,0.0
201,2023-11-07 15:00:00,4372.45,4391.60,4371.10,4389.13,16.49,0.377134,4392.0,4371.0,4389.0,...,20.50,4365.019,4237.75175,uptrend,4377.0,4162.0,215.0,8.675667,24,1.0
202,2023-11-07 16:00:00,4388.94,4392.40,4383.80,4386.33,-2.95,-0.067214,4392.0,4384.0,4386.0,...,8.60,4367.408,4238.73605,uptrend,4377.0,4163.0,214.0,8.662750,24,2.0
203,2023-11-07 17:00:00,4385.99,4385.99,4376.07,4384.21,-1.46,-0.033288,4386.0,4376.0,4384.0,...,9.92,4369.703,4239.77350,uptrend,4389.0,4163.0,226.0,8.695917,25,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5844,2024-10-24 12:00:00,5835.23,5837.15,5829.16,5830.31,-5.08,-0.087057,5837.0,5829.0,5830.0,...,7.99,5824.298,5846.69675,downtrend,5882.0,5784.0,98.0,8.855083,11,0.0
5845,2024-10-24 13:00:00,5830.15,5834.30,5810.67,5818.39,-11.90,-0.204111,5834.0,5811.0,5818.0,...,23.63,5825.339,5846.77515,downtrend,5882.0,5784.0,98.0,8.984417,10,0.0
5846,2024-10-24 14:00:00,5818.25,5824.02,5809.89,5812.82,-5.32,-0.091436,5824.0,5810.0,5813.0,...,14.13,5825.429,5846.76550,downtrend,5882.0,5784.0,98.0,9.010250,10,2.0
5847,2024-10-24 15:00:00,5812.93,5818.56,5795.13,5805.15,-7.62,-0.131087,5819.0,5795.0,5805.0,...,23.43,5825.024,5846.75150,downtrend,5882.0,5784.0,98.0,9.154167,10,1.0


c:\Users\Mritunjay Maddhesiya\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Mritunjay Maddhesiya\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\Mritunjay Maddhesiya\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [22]:

# #create trade logic
# def on_bar(data, trades, orders):
#     volume          = round(10000 / data['open'], 2) #1lots
#     open_trades     = trades[trades['state'] == 'open']
#     num_open_trades = open_trades.shape[0]

#     #entry signal
#     if data['open'] > data['period_high'] and data['trend'] == 'uptrend' and not num_open_trades:
#         tp = data['open'] + (data['open']-data['prev_low'])
#         orders.open_trade(symbol, volume, 'buy', sl=data['prev_low'], tp=tp)

#     #entry signal
#     elif data['open'] < data['period_low'] and data['trend'] == 'downtrend' and not num_open_trades:
#         tp = data['open'] + (data['open'] - data['prev_high'])
#         orders.open_trade(symbol, volume, 'sell', sl=data['prev_high'], tp=tp)



In [23]:
# #trade
# starting_balance = 10000
# currency = 'USD'
# exchange_rate = 1
# commission = -1

# #backtester
# bt = Backtester()
# bt.set_starting_balance(starting_balance, currency=currency)
# bt.set_exchange_rate(exchange_rate)
# bt.set_commission(commission)

# bt.set_historical_data(ohlc)
# bt.set_on_bar(on_bar)
# bt.run_backtest()

# bt.trades

In [24]:
# pnl_chart = bt.plot_pnl()
# pnl_chart

In [25]:
#evaluate_backtest(bt.trades)

In [26]:
def check_breakout(x):
    if x['open'] > x['period_high']:
        return 1 #bullish
    elif x['open'] < x['period_low']:
        return -1 #bearish
ohlc['breakout_type'] = ohlc.apply(check_breakout, axis=1)

breakout_df = ohlc[ohlc['breakout_type'].isin([1,-1])].copy()
breakout_df['count_breakouts'] = 1
display(breakout_df)

breakout_df2 = breakout_df.groupby(['breakout_type', 'hour'], as_index=False).agg({
    'perc_gain': 'sum',
    'count_breakouts': 'count'
})
pt.bar(breakout_df2, x='hour', y='perc_gain', barmode='group', color='breakout_type')

,time,open,high,low,close,gain,perc_gain,high_rounded,low_rounded,close_rounded,...,sma_200,trend,period_high,period_low,sr_range,atr,rel_sr_range,rel_candle_size,breakout_type,count_breakouts
202,2023-11-07 16:00:00,4388.94,4392.40,4383.80,4386.33,-2.95,-0.067214,4392.0,4384.0,4386.0,...,4238.73605,uptrend,4377.0,4163.0,214.0,8.662750,24,2.0,1.0,1
205,2023-11-07 19:00:00,4389.73,4391.79,4385.37,4386.29,-3.20,-0.072897,4392.0,4385.0,4386.0,...,4241.89425,uptrend,4389.0,4163.0,226.0,8.671750,26,1.0,1.0,1
223,2023-11-08 14:00:00,4389.50,4398.00,4389.43,4394.33,5.26,0.119831,4398.0,4389.0,4394.0,...,4261.83950,uptrend,4389.0,4180.0,209.0,7.904500,26,0.0,1.0,1
224,2023-11-08 15:00:00,4394.76,4395.03,4382.09,4382.59,-11.75,-0.267364,4395.0,4382.0,4383.0,...,4262.96765,uptrend,4389.0,4180.0,209.0,7.914000,26,1.0,1.0,1
243,2023-11-09 11:00:00,4399.39,4399.67,4391.48,4392.09,-7.24,-0.164568,4400.0,4391.0,4392.0,...,4284.67815,uptrend,4394.0,4251.0,143.0,7.197333,19,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,2024-10-15 04:00:00,5875.42,5875.49,5874.23,5874.75,-0.68,-0.011574,5875.0,5874.0,5875.0,...,5758.68690,uptrend,5875.0,5698.0,177.0,8.673917,20,0.0,1.0,1
5729,2024-10-17 11:00:00,5876.25,5876.25,5872.17,5873.32,-3.09,-0.052585,5876.0,5872.0,5873.0,...,5794.66810,uptrend,5875.0,5776.0,99.0,8.263250,11,0.0,1.0,1
5731,2024-10-17 13:00:00,5881.38,5889.64,5858.38,5866.30,-14.85,-0.252492,5890.0,5858.0,5866.0,...,5796.13915,uptrend,5876.0,5776.0,100.0,8.481917,11,1.0,1.0,1
5826,2024-10-23 17:00:00,5805.36,5805.36,5782.61,5783.60,-22.00,-0.378960,5805.0,5783.0,5784.0,...,5844.46670,downtrend,5882.0,5817.0,65.0,8.876667,7,2.0,-1.0,1


In [27]:

#create trade logic
def on_bar(data, trades, orders):
    volume          = round(10000 / data['open'], 2) #1lots

    open_trades     = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]

    #entry signal
    if data['breakout_type'] == 1 and data['hour'] in [17,18,19] and not num_open_trades:
        tp = data['open'] + (data['open'] - data['prev_low'])
        orders.open_trade(symbol, volume, 'buy')

    #entry signal
    elif data['breakout_type'] == -1 and data['hour'] in [17,18,19] and not num_open_trades:
        tp = data['open'] + (data['open'] - data['prev_high'])
        orders.open_trade(symbol, volume, 'sell')


    #exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]
        orders.close_trade(trade)


In [28]:
#trade
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = 0

#backtester
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)
bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,US500m,buy,2.28,2023-11-07 19:00:00,4389.73,2023-11-07 20:00:00,4386.53,0,0,{},-7.30,0.0,-7.3,-7.3,9992.7
1,closed,US500m,buy,2.27,2023-11-10 18:00:00,4402.42,2023-11-10 19:00:00,4410.05,0,0,{},17.32,0.0,17.32,10.02,10010.02
2,closed,US500m,buy,2.26,2023-11-13 17:00:00,4425.82,2023-11-13 18:00:00,4420.75,0,0,{},-11.46,0.0,-11.46,-1.44,9998.56
3,closed,US500m,buy,2.2,2023-11-20 17:00:00,4540.1,2023-11-20 18:00:00,4539.25,0,0,{},-1.87,0.0,-1.87,-3.31,9996.69
4,closed,US500m,buy,2.2,2023-11-20 19:00:00,4552.65,2023-11-20 20:00:00,4558.07,0,0,{},11.92,0.0,11.92,8.61,10008.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,closed,US500m,buy,1.75,2024-09-19 17:00:00,5727.95,2024-09-19 18:00:00,5737.32,0,0,{},16.40,0.0,16.4,198.49,10198.49
85,closed,US500m,buy,1.72,2024-10-11 19:00:00,5820.18,2024-10-13 22:00:00,5815.73,0,0,{},-7.65,0.0,-7.65,190.84,10190.84
86,closed,US500m,buy,1.71,2024-10-14 17:00:00,5864.6,2024-10-14 18:00:00,5862.75,0,0,{},-3.16,0.0,-3.16,187.68,10187.68
87,closed,US500m,buy,1.7,2024-10-14 19:00:00,5869.26,2024-10-14 20:00:00,5870.26,0,0,{},1.70,0.0,1.7,189.38,10189.38


In [29]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [30]:
breakout_df3 = breakout_df.groupby(['breakout_type', 'rel_candle_size', 'hour'], as_index=False).agg({
    'perc_gain': 'sum',
    'count_breakouts': 'count'
})
pt.bar(breakout_df3, x='rel_candle_size', y='perc_gain', barmode='group', color='breakout_type', hover_data=['count_breakouts', 'hour'])